In [1]:
import pandas as pd
from joblib import dump
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np


C:\Users\86188\AppData\Local\Temp\ipykernel_34656\1346217225.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
genes_train = pd.read_csv("https://cs307.org/lab-05/data/genes-train.csv")
# create X and y for train
X_train = genes_train.drop("y", axis=1)
y_train = genes_train["y"]

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np

# 定义包含特征工程步骤和Lasso回归的管道
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 数据标准化
    ('poly', PolynomialFeatures()),  # 生成多项式特征
    ('lasso', Lasso(max_iter=10000))  # Lasso回归
])

# 设置参数网格，包括Lasso的alpha参数和多项式特征的度数
param_grid = {
    'poly__degree': [1, 2],  # 尝试1阶、2阶和3阶多项式特征
    'lasso__alpha': np.logspace(-3, 1, 100),  # Lasso的alpha参数范围
    # 可以根据需要添加其他参数
}

# 创建GridSearchCV实例
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# 在训练数据上执行网格搜索
grid_search.fit(X_train, y_train)

# 获取最佳参数和对应的性能指标
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# 输出最佳参数
print(f"Best parameters: {best_params}")

# 输出最佳模型的RMSE
best_rmse = np.sqrt(-best_score)
print(f"Best RMSE: {best_rmse}")


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Best parameters: {'lasso__alpha': 0.01788649529057435, 'poly__degree': 1}
Best RMSE: 0.32235601585695545


In [4]:
dump(grid_search, "gene-expression.joblib")

['gene-expression.joblib']